# **Main ICM Final Project Jupyter Notebook**

## **Justification for Choice of Images**

We chose three qualitatively different images:

- `antimeme.png`: A vector-like, clean image with large flat color regions and strong contrast. This makes it easier to visually assess blur and regularization artifacts, although the small “THIS IS FINE” text is very sensitive to blur.

- `save_me.png`: A real-life photo with natural lighting, surface texture, sensor noise, and overlaid Chinese text plus artificial purple scribble. These factors introduce mixed frequencies and uneven illumination.

- `this_is_fine.png`: A comic drawing that is not vectorized and contains bold ink lines, but also visible grain and scan noise. It lies between the other two in complexity: more texture than `antimeme.png` but cleaner than the real photo `save_me.png`.

**At first**, we hypothesized that deconvolution difficulty would follow `save_me.png` > `this_is_fine.png` > `antimeme.png`, because natural photos usually contain more complex textures and noise.  

However, the experiments in the Results section will show even though the overall error may be smaller, the tiny high-frequency text "THIS IS FINE" and smaller details of the main subject in `antimeme.png` actually are the hardest to recover under strong blur. This shows, actually, shadows and variation in images aren't the hardest part, but rather the size of the subjects at hand --- even vector-like components that are small are very hard to recover. Moreover, this shows how quantitative results such as MSE and PSNR may not accurately depict the perceptive effectiveness or meaningful image recovery.

## **Justification for Choice of Private Key Image**

`private_key.png` is a real-life photo, which means the natural lighting may create enough noise. As the photo subject is mainly green, this will ensure the kernels for R, G, and B will be different. All in all, these factors should make it a less predictable kernel. However, it shouldn't be too unpredictable to the point where recovery is hard.

## <u>**Experiment Plan**</u>

## **Experiments**

Repeat the following kernels for all three images. We consider R, G, B as three separate channels needed for kernels and do not do grayscale, for better visual differentiation.

### **Baseline: "Transformations" (No Private Key Convolutions)**
 - Gaussian Blur
 
 - Motion Blur

 - Box Blur

### **New: "Encryptions" (Private Key Convolutions)**
 - Random Kernel from a seed (seed = private key), of course fix the seed across all three images
 
 - Private Key Image-derived Kernel via `private_key.png`

## **Data Extracted**

For each image + kernel + parameter setting, store:

### Quantitative Data
 - **Image ID**: `antimeme`, `this_is_fine`, `save_me`
 
 - **Kernel type**: `Gaussian`, `Motion`, `Box`, `RandomSeed`, `KeyImage`
 
 - **Kernel parameters**: size, $\sigma$ (for Gaussian), length/angle (for motion), seed, which key image
 - **Noise level** $\sigma$ (for added Gaussian noise)
 
 - **Regularization parameter** $\lambda$

 - **MSE** between original and recovered (over all RGB channels)
 
 - **PSNR** derived from MSE

### Qualitative Data
 - **The Images**: original vs encrypted vs noisy vs recovered
 
 - Notes on **visual artifacts** such as ringing around edges, halo artifacts, oversmoothing, color shifts, "ghosting" shapes, complete failure (looks random)

## **Methodology**

### Summary
This project studies how image reconstruction behaves under different convolution-based transformations, including both standard (public) kernels and “encrypted” private-key kernels. We evaluate reconstruction accuracy when the correct kernel is known, and investigate the sensitivity of the inverse problem by testing recovery with intentionally incorrect kernels. We do not perform blind kernel estimation; all recovery assumes access to either the true kernel or a deliberately mismatched one.

### 1. Preprocessing
- Load each image (`antimeme`, `this_is_fine`, `save_me`) as an RGB array of size $512 \times 512 \times 3$.
- Normalize pixel values to $[0,1]$.
- No grayscale conversion is performed; convolution is applied independently to each R, G, B channel.

### 2. Apply Operator (Forward Model)
For each image and each kernel type:

#### Baseline Transformations (public kernels)
These kernels are fully known:
- Gaussian blur (fixed size and $\sigma$)
- Motion blur (fixed length and angle)
- Box blur

#### Encryption Operators (private-key kernels)
These kernels are not publicly known:
- Random kernel generated from a fixed seed (seed = private key)
- Key-image-derived kernel from `private_key.png`

Each kernel is applied channel-wise to form:
$$
y = Kx
$$

Then additive Gaussian noise is applied:
$$
z = y + \text{noise}
$$

Noise is added to make the inverse problem realistic.

### 3. Recovery (Inverse Problem)

#### Correct-Kernel Recovery
For each baseline or private-key kernel, perform non-blind deconvolution by solving:

$$
\hat{x} = \arg\min_x \|Kx - z\|^2 + \lambda \|x\|^2
$$

where:
- $K$ = the known convolution operator
- $\lambda$ = regularization strength

The solution is computed channel-wise using:
- an FFT-based approximate inverse.

#### Incorrect-Kernel Recovery (for encryption tests)
To demonstrate key-dependence, we intentionally recover using a wrong kernel:
- wrong random seed
- wrong kernel size
- Gaussian or motion blur instead of the private kernel

We solve the same inverse problem using $\tilde{K}$:

$$
\hat{x}_{wrong} = \arg\min_x \|\tilde{K}x - z\|^2 + \lambda \|x\|^2
$$

This typically produces severe reconstruction failure and demonstrates that successful recovery requires the correct key.

We do not attempt to guess or estimate kernels.

### 4. Evaluation

#### Quantitative Metrics
- MSE between original and recovered RGB image:
$$
\text{MSE} = \frac{1}{N} \sum_{i=1}^N (x_i - \hat{x}_i)^2
$$

- PSNR derived from MSE:
$$
\text{PSNR} = 10 \log_{10} \left( \frac{1}{\text{MSE}} \right)
$$

(assuming images are normalized to $[0,1]$).

#### Recorded Metadata
For each experiment, record:
- kernel type
- kernel parameters
- noise level $\sigma$
- regularization $\lambda$
- whether the kernel used for recovery was correct or wrong

#### Qualitative Comparison
For each experiment, visually compare:
- original image
- transformed/encrypted image
- noisy observation
- recovered image (correct key)
- recovered image (wrong key)

and note artifacts such as ringing, oversmoothing, color distortion, or complete failure.

## **Mathematical Formulation of the Code Implementation**

### 0. Notation

We model a color image as a discrete RGB function
$$
x(i,j,c), \quad i,j \in \{0,\dots,H-1\}, \; c \in \{R,G,B\}.
$$

In this project we fix
$$
H = W = 512,
$$
so each image is of size $512 \times 512 \times 3$, and all pixel values are normalized to the range $[0,1]$.

A (possibly color-dependent) convolution kernel is denoted by
$$
k_c(u,v), \quad u,v \in \{-r,\dots,r\},
$$
where $c$ is the color channel and the kernel size is $(2r+1) \times (2r+1)$.

The 2D discrete convolution of an image channel $x_c$ with kernel $k_c$ is
$$
(K_c x_c)(i,j)
= \sum_{u=-r}^{r} \sum_{v=-r}^{r} k_c(u,v)\, x_c(i-u,\, j-v),
$$
with indices handled by padding or circular wrapping in the code.

For a kernel that is the same across channels (Gaussian, box, motion, random), we write $k(u,v)$ and
$$
(Kx)(i,j,c) = \sum_{u,v} k(u,v)\, x(i-u, j-v, c).
$$

---

### 1. Convolution Kernels (corresponding to `kernels.py`)

#### 1.1 Box Blur Kernel

For a box blur of size \(s \times s\), the kernel is constant:
$$
k_{\text{box}}(u,v) = \frac{1}{s^2}, \quad u,v \in \left\{ -\frac{s-1}{2}, \dots, \frac{s-1}{2} \right\}.
$$

This corresponds to uniform averaging over an \(s \times s\) neighborhood.

#### 1.2 Gaussian Blur Kernel

For a Gaussian blur of size $s \times s$ and standard deviation $\sigma$, we define
$$
g(u,v) = \exp\!\left( -\frac{u^2 + v^2}{2\sigma^2} \right),
$$
for $u,v$ in the appropriate index range, and then normalize:
$$
k_{\text{gauss}}(u,v) = \frac{g(u,v)}{\sum_{u',v'} g(u',v')}.
$$

This ensures
$$
\sum_{u,v} k_{\text{gauss}}(u,v) = 1,
$$
so constant images remain constant under the blur.

#### 1.3 Motion Blur Kernel

A motion blur kernel of length $L$ and angle $\theta$ is represented as a line of ones along a direction given by $\theta$, normalized by its sum. Conceptually, in an $L \times L$ grid:
- initialize a line along the horizontal center row:
  $$
  k_0(u,v) =
  \begin{cases}
  1, & \text{if } u = 0 \text{ and } v \in \{-\tfrac{L-1}{2},\dots,\tfrac{L-1}{2}\}, \\
  0, & \text{otherwise},
  \end{cases}
  $$
- rotate this pattern by angle $\theta$,
- then normalize:
  $$ 
  k_{\text{motion}}(u,v) = \frac{k_{\text{rot}}(u,v)}{\sum_{u',v'} k_{\text{rot}}(u',v')}.
  $$

This models a uniform motion along a line.

#### 1.4 Random Kernel (Seeded)

Let $s$ be the kernel size and $\text{seed}$ a fixed integer. The kernel entries are generated as i.i.d. random values, then normalized:
$$
\tilde{k}(u,v) \sim \text{Uniform}(0,1),
$$
$$
k_{\text{rand}}(u,v) = \frac{\tilde{k}(u,v)}{\sum_{u',v'} \tilde{k}(u',v')}.
$$

The fixed seed makes this kernel deterministic and repeatable, so it behaves as a "private key" known only to the experimenter.

#### 1.5 RGB Key-Image-Derived Kernel

Let the private key image be
$$
x_{\text{key}}(i,j,c), \quad c \in \{R,G,B\}.
$$

For each channel $c$, we:
1. Resize $x_{\text{key}}(\cdot,\cdot,c)$ to a small kernel size $s \times s$,
2. Normalize it to sum to 1:
   $$
   k_c(u,v) = \frac{x_{\text{key, resized}}(u,v,c)}{\sum_{u',v'} x_{\text{key, resized}}(u',v',c)}.
   $$

This defines a **channel-dependent kernel**:
$$
K_c x_c = k_c * x_c,
$$
and the overall forward operator is
$$
(Kx)(i,j,c) = \sum_{u,v} k_c(u,v)\, x(i-u,j-v,c).
$$

This construction makes the encryption depend on the color structure of the private key image.

---

### 2. Forward Model (corresponding to `apply_kernel_fft`)

For a given kernel (or set of kernels) $K$ and original image $x$, the noiseless forward model is
$$
y = Kx,
$$
meaning, per channel,
$$
y_c = K_c x_c = k_c * x_c.
$$

Additive Gaussian noise is then applied:
$$
z = y + \eta,
$$
where
$$
\eta(i,j,c) \sim \mathcal{N}(0, \sigma_{\text{noise}}^2)
$$
independently for each pixel and channel. The observed image is thus
$$
z(i,j,c) = (K_c x_c)(i,j) + \eta(i,j,c).
$$

In code, the forward convolution is implemented via FFT using the convolution theorem.

---

### 3. FFT-Based Convolution and the Convolution Theorem (used in `fft_deconv.py`)

For a 2D discrete signal $x(i,j)$ and kernel $k(u,v)$, define their 2D discrete Fourier transforms:
$$
\hat{x}(\omega_1,\omega_2) = \mathcal{F}\{x\}(\omega_1,\omega_2),
$$
$$
\hat{k}(\omega_1,\omega_2) = \mathcal{F}\{k\}(\omega_1,\omega_2).
$$

The convolution theorem states:
$$
\mathcal{F}\{k * x\}(\omega_1,\omega_2) = \hat{k}(\omega_1,\omega_2)\, \hat{x}(\omega_1,\omega_2).
$$

In the code, this is implemented by:
1. Padding the kernel \(k\) to the same spatial size as the image,
2. Taking FFTs:
   $$
   \hat{K} = \mathcal{F}\{k_{\text{padded}}\}, \quad
   \hat{X}_c = \mathcal{F}\{x_c\},
   $$
3. Multiplying in the frequency domain:
   $$
   \hat{Y}_c = \hat{K} \cdot \hat{X}_c,
   $$
4. Inverse FFT:
   $$
   y_c = \mathcal{F}^{-1}\{\hat{Y}_c\}.
   $$

This is what `apply_kernel_fft` does for each channel.

---

### 4. Regularized Inverse (Deconvolution) — `deconv_fft`

#### 4.1 Tikhonov-Regularized Minimization Problem

Given the observed image $z$ and known operator $K$, we solve a Tikhonov-regularized least squares problem:
$$
\hat{x} = \arg\min_x \|Kx - z\|^2 + \lambda \|x\|^2,
$$
where $\lambda > 0$ is the regularization strength. For simplicity, we take $L = I$, so the penalty term is $\|x\|^2$.

The normal equations for this problem are:
$$
(K^T K + \lambda I)\hat{x} = K^T z.
$$

Because $K$ is a convolution operator (assumed circulant after padding), it is diagonalized by the FFT.

#### 4.2 Frequency-Domain Derivation

Let $\hat{K}(\omega_1,\omega_2)$ and $\hat{Z}(\omega_1,\omega_2)$ be the FFTs of the kernel and the observed image channel. In the frequency domain, the normal equations become, for each frequency $(\omega_1, \omega_2)$:
$$
(|\hat{K}(\omega)|^2 + \lambda)\, \hat{X}(\omega)
= \overline{\hat{K}(\omega)}\, \hat{Z}(\omega),
$$
so
$$
\hat{X}(\omega)
= \frac{\overline{\hat{K}(\omega)}\, \hat{Z}(\omega)}{|\hat{K}(\omega)|^2 + \lambda}.
$$

This is the standard Tikhonov-regularized deconvolution (Wiener-like) formula.

The recovered image channel is obtained by inverse FFT:
$$
\hat{x}(i,j) = \mathcal{F}^{-1}\{\hat{X}(\omega)\}(i,j).
$$

In the RGB case, this is applied independently to each channel:
$$
\hat{X}_c(\omega)
= \frac{\overline{\hat{K}_c(\omega)}\, \hat{Z}_c(\omega)}{|\hat{K}_c(\omega)|^2 + \lambda},
$$
$$
\hat{x}_c = \mathcal{F}^{-1}\{\hat{X}_c\}.
$$

This is exactly what `deconv_fft` (and its RGB wrapper) implements.

---

### 5. Wrong-Kernel Recovery — Key Dependence

For encryption experiments, the true forward operator is $K$, but we deliberately use an *incorrect* kernel $\tilde{K}$ in the inverse:
$$
\hat{x}_{\text{wrong}} = \arg\min_x \|\tilde{K}x - z\|^2 + \lambda \|x\|^2,
$$
leading in the frequency domain to
$$
\hat{X}_{\text{wrong}}(\omega)
= \frac{\overline{\hat{\tilde{K}}(\omega)}\, \hat{Z}(\omega)}{|\hat{\tilde{K}}(\omega)|^2 + \lambda}.
$$

Since $\tilde{K} \neq K$, this corresponds to solving the wrong inverse problem and generally yields severe reconstruction artifacts or failure. This behavior demonstrates the dependence of successful recovery on the correct "key" (kernel).

---

### 6. Error Metrics — `metrics.py`

#### 6.1 Mean Squared Error (MSE)

For two RGB images $x$ and $\hat{x}$ of size $H \times W \times 3$, we flatten all channels and pixels into one vector of length $N = H \cdot W \cdot 3$, and define
$$
\text{MSE}(x,\hat{x}) = \frac{1}{N} \sum_{i=1}^{N} (x_i - \hat{x}_i)^2.
$$

This is implemented as `np.mean((a - b)**2)`.

#### 6.2 Peak Signal-to-Noise Ratio (PSNR)

Assuming images are normalized to the range $[0,1]$, the peak possible value is
$$
\text{MAX} = 1.
$$

Then PSNR is defined as
$$
\text{PSNR} = 10 \log_{10} \left( \frac{\text{MAX}^2}{\text{MSE}} \right)
= 10 \log_{10} \left( \frac{1}{\text{MSE}} \right).
$$

This is implemented as `10 * np.log10(1.0 / mse_value)`.

High PSNR corresponds to good reconstruction; very low PSNR corresponds to failure, especially in the wrong-key experiments.

---